## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json

# Import API key
from config import g_API_KEY
# import os
# from dotenv import load_dotenv
#Google_API_KEY = os.getenv("g_API_KEY")



In [2]:
# Configure gmaps API key
gmaps.configure(api_key= g_API_KEY)


# why Unnamed column?

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
# print(city_data_df.columns)
# city_data_df = city_data_df.set_index('Unnamed: 0')
city_data_df.drop(columns="Unnamed: 0", inplace=True)
# city_data_df.drop(index = 0, inplace=True) # use onl.y to remove rows
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Puerto Ayora,EC,-0.7393,-90.3518,74.55,97,72,0.47,broken clouds
1,Constitucion,CL,-35.3333,-72.4167,58.64,92,66,1.77,broken clouds
2,Cidreira,BR,-30.1811,-50.2056,73.67,76,100,14.14,light rain
3,Carnarvon,AU,-24.8667,113.6333,86.07,54,40,24.16,scattered clouds
4,Punta Arenas,CL,-53.1500,-70.9167,46.51,71,20,19.57,few clouds


In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv", index_col = 0)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Puerto Ayora,EC,-0.7393,-90.3518,74.55,97,72,0.47,broken clouds
1,Constitucion,CL,-35.3333,-72.4167,58.64,92,66,1.77,broken clouds
2,Cidreira,BR,-30.1811,-50.2056,73.67,76,100,14.14,light rain
3,Carnarvon,AU,-24.8667,113.6333,86.07,54,40,24.16,scattered clouds
4,Punta Arenas,CL,-53.1500,-70.9167,46.51,71,20,19.57,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
Min = float(input("What is the minimum temperature you would like for your trip? "))
Max = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 77


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filter_city_df = city_data_df.loc[(city_data_df['Max Temp']<=Max)
                                 & (city_data_df['Max Temp']>=Min)]
print(len(filter_city_df))
filter_city_df.head()


23


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,Busselton,AU,-33.6500,115.3333,75.60,60,90,14.65,overcast clouds
114,Arraial Do Cabo,BR,-22.9661,-42.0278,76.55,91,7,11.68,clear sky
125,Chepareria,KE,1.3077,35.2039,76.53,35,100,3.85,overcast clouds
150,Port Macquarie,AU,-31.4333,152.9167,76.89,74,57,7.76,broken clouds
182,Vila Velha,BR,-20.3297,-40.2925,75.94,91,75,3.44,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
filter_city_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 6 to 659
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 23 non-null     object 
 1   Country              23 non-null     object 
 2   Lat                  23 non-null     float64
 3   Lng                  23 non-null     float64
 4   Max Temp             23 non-null     float64
 5   Humidity             23 non-null     int64  
 6   Cloudiness           23 non-null     int64  
 7   Wind Speed           23 non-null     float64
 8   Current Description  23 non-null     object 
dtypes: float64(4), int64(2), object(3)
memory usage: 1.8+ KB


In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filter_city_df.dropna()

clean_df.head()


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,Busselton,AU,-33.6500,115.3333,75.60,60,90,14.65,overcast clouds
114,Arraial Do Cabo,BR,-22.9661,-42.0278,76.55,91,7,11.68,clear sky
125,Chepareria,KE,1.3077,35.2039,76.53,35,100,3.85,overcast clouds
150,Port Macquarie,AU,-31.4333,152.9167,76.89,74,57,7.76,broken clouds
182,Vila Velha,BR,-20.3297,-40.2925,75.94,91,75,3.44,broken clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Busselton,AU,75.60,overcast clouds,-33.6500,115.3333,
114,Arraial Do Cabo,BR,76.55,clear sky,-22.9661,-42.0278,
125,Chepareria,KE,76.53,overcast clouds,1.3077,35.2039,
150,Port Macquarie,AU,76.89,broken clouds,-31.4333,152.9167,
182,Vila Velha,BR,75.94,broken clouds,-20.3297,-40.2925,
188,Bathsheba,BB,76.71,scattered clouds,13.2167,-59.5167,
215,Ewa Beach,US,76.98,clear sky,21.3156,-158.0072,
275,Ahuimanu,US,76.75,broken clouds,21.4447,-157.8378,
350,Buchanan,LR,76.75,broken clouds,5.8808,-10.0467,
370,Ballina,AU,76.01,broken clouds,-28.8667,153.5667,


# **TEST**

In [10]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_API_KEY,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEDv_zmOmyFgIM9XIJNUOGl6OKzzmrBYw70pWVzN3r5BWwfn8o2tCjoKRYuNSpFQd9CoI7GP-9oaIHIuhHKY2_qqY225gn_6pB4NrJgWdKscWDjvDgSPlEHX54Ek8b-C4O08Z66fimKrSjCimenhuv-owbpa3rJQdE6BdN3pYs-U-qH1MHCbTiObIono7myXCdxaA0GVJllb3XeEeyTYuKcVD82SXgYeBOo9N2C0bJx0jykmmPOkvkJeQ3gd0Js6amREbe5QX9NRjpPgtPsb_6bJkfO5Sn9BIkma8LezXwdR2_g55OVqeJZiQs2HXyV7PkrHEjl3tUjL4LXOchlUoVPF0IofDmbDyFX6zrNN7oYn2Hd9S-fXcRsAe9i5fqtp2Ndr3qld_HzYddyLjsjWAIs-6TePjEf4P-DOCc0la5LON_wuJE9yU55K',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [11]:
lat=24.8667
lng=113.6333
params = {
    "location": f'{lat}, {lng}',
    "radius": 5000,
    "type": "lodging",
    "key": g_API_KEY
}

base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels= requests.get(base_url, params=params).json()
print(json.dumps(hotels, indent=4))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 24.835272,
                    "lng": 113.608783
                },
                "viewport": {
                    "northeast": {
                        "lat": 24.8365917302915,
                        "lng": 113.6100539302915
                    },
                    "southwest": {
                        "lat": 24.8338937697085,
                        "lng": 113.6073559697085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "\u78a7\u6842\u56ed\u51e4\u51f0\u9152\u5e97",
            "photos": [
                {
              

In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_API_KEY
}

# 6b. Iterate through the hotel DataFrame.
for i, city in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = city["Lat"]
    lng = city["Lng"]
    params["location"] = f'{lat}, {lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Processing | {city['City']}")
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels= requests.get(base_url, params=params).json()
    #print(json.dumps(hotels, indent=4))
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, 'Hotel Name']= hotels['results'][0]['name']
    except:
        print(f"Hotel not found. Skipping {city['City']}...")

print("------Completed-------")

Processing | Busselton
Processing | Arraial Do Cabo
Processing | Chepareria
Processing | Port Macquarie
Processing | Vila Velha
Processing | Bathsheba
Processing | Ewa Beach
Processing | Ahuimanu
Processing | Buchanan
Processing | Ballina
Processing | Cabinda
Processing | Oistins
Processing | Boa Vista
Processing | Makakilo City
Processing | Belmonte
Processing | Lahij
Hotel not found. Skipping Lahij...
Processing | Otukpo
Processing | Libreville
Processing | Guatire
Processing | Whyalla
Processing | Illela
Hotel not found. Skipping Illela...
Processing | Dunmore Town
Processing | Acapulco
------Completed-------


In [14]:
print(len(hotel_df))
hotel_df

23


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Busselton,AU,75.60,overcast clouds,-33.6500,115.3333,Observatory Guest House
114,Arraial Do Cabo,BR,76.55,clear sky,-22.9661,-42.0278,Pousada Porto Praia
125,Chepareria,KE,76.53,overcast clouds,1.3077,35.2039,Hamza Lodge
150,Port Macquarie,AU,76.89,broken clouds,-31.4333,152.9167,Rydges Port Macquarie
182,Vila Velha,BR,75.94,broken clouds,-20.3297,-40.2925,Hotel Vitória Palace
188,Bathsheba,BB,76.71,scattered clouds,13.2167,-59.5167,Atlantis Hotel
215,Ewa Beach,US,76.98,clear sky,21.3156,-158.0072,Base Lodging
275,Ahuimanu,US,76.75,broken clouds,21.4447,-157.8378,Paradise Bay Resort
350,Buchanan,LR,76.75,broken clouds,5.8808,-10.0467,Kamaneahn Hotel Inc
370,Ballina,AU,76.01,broken clouds,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron


In [15]:
# 7. Drop the rows where there is no Hotel Name.
#clean_hotel_df = hotel_df.dropna()
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]

print(len(clean_hotel_df))

clean_hotel_df

21


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Busselton,AU,75.60,overcast clouds,-33.6500,115.3333,Observatory Guest House
114,Arraial Do Cabo,BR,76.55,clear sky,-22.9661,-42.0278,Pousada Porto Praia
125,Chepareria,KE,76.53,overcast clouds,1.3077,35.2039,Hamza Lodge
150,Port Macquarie,AU,76.89,broken clouds,-31.4333,152.9167,Rydges Port Macquarie
182,Vila Velha,BR,75.94,broken clouds,-20.3297,-40.2925,Hotel Vitória Palace
188,Bathsheba,BB,76.71,scattered clouds,13.2167,-59.5167,Atlantis Hotel
215,Ewa Beach,US,76.98,clear sky,21.3156,-158.0072,Base Lodging
275,Ahuimanu,US,76.75,broken clouds,21.4447,-157.8378,Paradise Bay Resort
350,Buchanan,LR,76.75,broken clouds,5.8808,-10.0467,Kamaneahn Hotel Inc
370,Ballina,AU,76.01,broken clouds,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [18]:
# 11a. Add a marker layer for each city to the map. 
#max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30,31), zoom_level=1.5)
#heat_layer= gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))